# Import Schools Data
Import data from https://get-information-schools.service.gov.uk/

## Setup

### Parameters

In [1]:
pFilepath = "Files"
debug_yn = False

StatementMeta(, cba401c0-961c-438f-8aa6-49f9cf31292d, 3, Finished, Available, Finished)

### Imports

In [2]:
import logging
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
from notebookutils import mssparkutils
from pyspark.sql import DataFrame

StatementMeta(, cba401c0-961c-438f-8aa6-49f9cf31292d, 4, Finished, Available, Finished)

### Functions

In [3]:
# More Pythonic method but ended up with duplicate columns
# Function to clean up column names
def clean_column_names(df: DataFrame) -> DataFrame:
    # Create a mapping of original column names to cleaned column names: \W+ matches one or more non-word characters (equivalent to [^a-zA-Z0-9_])
    cleaned_columns = {col: re.sub(r'\W+', '_', col).strip('_').lower() for col in df.columns}

    # Rename columns using the mapping
    return df.toDF(*[cleaned_columns[col] for col in df.columns])

StatementMeta(, cba401c0-961c-438f-8aa6-49f9cf31292d, 5, Finished, Available, Finished)

In [4]:
def split_date_from_filename(table_name):
    """
    Cleans the table name by removing the date in yyyyMMdd format if present at the end.

    :param table_name: The original table name.
    :return: The cleaned table name with the date removed.
    """
    # Regular expression to match the date in the format yyyyMMdd at the end of the table name
    cleaned_name = re.sub(r'\d{8}$', '', table_name)
    
    # Optionally, can clean up any trailing underscores or other characters
    cleaned_name = cleaned_name.strip('_').lower()
    
    return cleaned_name

StatementMeta(, cba401c0-961c-438f-8aa6-49f9cf31292d, 6, Finished, Available, Finished)

## Main

### Drop existing tables
Drops all existing tables in the Lakehouse

In [5]:
df = spark.sql("SHOW TABLES")
display(df)

# Drop all tables
for table in df.select("tableName").collect():
    try:
        if debug_yn:
            print(f"Would drop table: {table['tableName']}")
        else:
            spark.sql(f"DROP TABLE {table['tableName']}")
            print(f"Dropped table: {table['tableName']}")
    except Exception as e:
        print(f"Failed to drop table: {table['tableName']} due to error: {e}")
        
print(" ")
print("---- DROP TABLES END ----")

StatementMeta(, cba401c0-961c-438f-8aa6-49f9cf31292d, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c5c40f9a-aef8-4e14-9f6e-505bd44c49c7)

 
---- DROP TABLES END ----


### Get List of files

In [6]:
# Get list of files to load - can't use wildcards here
files = mssparkutils.fs.ls(f"{pFilepath}")

# Filter the list to include only CSV files
csv_files = [file for file in files if file.name.endswith('.csv')]

print(f"Total no. of files: {len(files)} -> Filtered files: {len(csv_files)}")

StatementMeta(, cba401c0-961c-438f-8aa6-49f9cf31292d, 8, Finished, Available, Finished)

Total no. of files: 17 -> Filtered files: 16


### Load the files

In [7]:
# Loop through the files and load them (serial)
for file in csv_files:
    # Remove the .csv element from filename to make a clean table name
    clean_tablename = file.name.split('.csv')[0]
    
    # Split date from filename
    clean_tablename = split_date_from_filename(clean_tablename)

    print(f"Processing file: {file.name} -> Clean table name: {clean_tablename}")
  
    # Load the table to a dataframe
    df = spark.read.format("csv").option("header","true").load(f"{pFilepath}/{file.name}")
    
    # Clean up the column names
    df = clean_column_names(df)

    # Save to the database
    df.write.format("delta").mode("overwrite").save(f"Tables/{clean_tablename}")

print(" ")
print("---- LOAD TABLES END ----")

StatementMeta(, cba401c0-961c-438f-8aa6-49f9cf31292d, 9, Finished, Available, Finished)

Processing file: academiesmatmembership20250106.csv -> Clean table name: academiesmatmembership
Processing file: allgroupsdata20250106.csv -> Clean table name: allgroupsdata
Processing file: alllinksdata20250106.csv -> Clean table name: alllinksdata
Processing file: edubaseallacademiesandfree20250106.csv -> Clean table name: edubaseallacademiesandfree
Processing file: edubaseallchildrencentre20250106.csv -> Clean table name: edubaseallchildrencentre
Processing file: edubasealldata20250106.csv -> Clean table name: edubasealldata
Processing file: edubaseallstatefunded20250106.csv -> Clean table name: edubaseallstatefunded
Processing file: governanceacaddata20250106.csv -> Clean table name: governanceacaddata
Processing file: governancealldata20250106.csv -> Clean table name: governancealldata
Processing file: governanceladata20250106.csv -> Clean table name: governanceladata
Processing file: governancematdata20250106.csv -> Clean table name: governancematdata
Processing file: grouplinks_